<a href="https://colab.research.google.com/github/1uch0/LLM_Udemy_course/blob/main/Chatbot_inA_Manufacturing_Environment_asking_questions_about_assets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#!pip install gradio
import os
from openai import OpenAI
from google.colab import userdata

In [8]:
import gradio as gr
import json
import os
import openai


In [10]:

# OpenAI API Key (Replace with your own key)

api_key = userdata.get('Open_AI_API') ##Open_AI_API It is the API key from OPEN_AI saved in your collab
openai = OpenAI(api_key=api_key)
openai_api_key = api_key

# File to store robot resources
FILE_PATH = "robot_resources.json"

# Load existing robots from file
def load_robots():
    if os.path.exists(FILE_PATH):
        with open(FILE_PATH, "r") as file:
            return json.load(file)
    return []

# Save robots to file
def save_robots():
    with open(FILE_PATH, "w") as file:
        json.dump(robot_resources, file, indent=2)

# Dictionary to store robot resources
robot_resources = load_robots()

# Function to add a robot
def add_robot(name, type, capabilities):
    robot = {"name": name, "type": type, "capabilities": capabilities}
    robot_resources.append(robot)
    save_robots()
    return f"Robot {name} added successfully!"

# Format robots for display
def format_robots(robots):
    if not robots:
        return "No robots found."
    return "\n\n".join(
        [f"**Name:** {r['name']}  \n**Type:** {r['type']}  \n**Capabilities:** {', '.join(r['capabilities'])}" for r in robots]
    )

# Function to retrieve all robots
def view_robots():
    return json.dumps(robot_resources, indent=2)

# Function to filter robots by capability
def find_robots_by_capability(capability):
    filtered_robots = [robot for robot in robot_resources if capability in robot["capabilities"]]
    return json.dumps(filtered_robots, indent=2) if filtered_robots else "No robots found with that capability."

# Function to process OpenAI query
def ask_openai(prompt):
    messages=[{"role": "system", "content": "You are an expert in robotics resource management."},
                  {"role": "user", "content": f"{prompt}\n\nHere is the stored robot data:\n{json.dumps(robot_resources, indent=2)}"}]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,

    )
    return response.choices[0].message.content

# Gradio UI
def launch_app():
    robot_types = ["AGV", "CNC", "Robot"]
    capabilities_list = ["Transport", "Milling", "Welding", "Pick and Place", "Painting"]

    with gr.Blocks() as demo:
        gr.Markdown("## Robot Resource Manager")

        with gr.Row():
            name = gr.Textbox(label="Robot Name")
            type = gr.Dropdown(choices=robot_types, label="Robot Type")
            capabilities = gr.CheckboxGroup(choices=capabilities_list, label="Capabilities")
            add_button = gr.Button("Add Robot")

        status_output = gr.Textbox(label="Status", interactive=False)
        add_button.click(add_robot, inputs=[name, type, capabilities], outputs=[status_output])

        view_button = gr.Button("View All Robots")
        robots_output = gr.Markdown()
        view_button.click(view_robots, inputs=[], outputs=[robots_output])

        capability_input = gr.Dropdown(choices=capabilities_list, label="Find Robots by Capability")
        capability_button = gr.Button("Find")
        capability_output = gr.Markdown()
        capability_button.click(find_robots_by_capability, inputs=[capability_input], outputs=[capability_output])

        gr.Markdown("## Ask about Robots")
        openai_prompt = gr.Textbox(label="Ask anything about the stored robots")
        openai_button = gr.Button("Ask AI")
        openai_output = gr.Textbox(label="AI Response", interactive=False)
        openai_button.click(ask_openai, inputs=[openai_prompt], outputs=[openai_output])

    demo.launch(share=True, debug=True )

# Launch Gradio in Colab
launch_app()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e4324529317c0b34de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e4324529317c0b34de.gradio.live
